# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Загрузим нужные библиотеки

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Сохраним данные в переменные

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Напишем функцию для отображения основной информации

In [3]:
def information(list):
    for index, i in enumerate(list, start=0):
        print('*'*20, f'Таблица geo_data_{index}', '*'*20)
        print(i.head())
        print('-'*60)
        i.info()
        print('-'*60)
        print(i.describe())
        print('-'*60)
        print('Дубликатов в таблице -', i.duplicated().sum())
        print('\n')

Применим функцию к таблицам

In [4]:
information([geo_data_0, geo_data_1, geo_data_2])

******************** Таблица geo_data_0 ********************
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
------------------------------------------------------------
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.

Дубликатов в данных не обнаружено, о "нормальности" значений признаков мы судить не можем

Заменим индексы на id скважин

In [5]:
geo_data_0 = geo_data_0.set_index('id')
geo_data_1 = geo_data_1.set_index('id')
geo_data_2 = geo_data_2.set_index('id')

## Обучение и проверка модели

Разделим данные на признаки и целевые признаки

In [6]:
features_0 = geo_data_0.drop('product', axis=1)
features_1 = geo_data_1.drop('product', axis=1)
features_2 = geo_data_2.drop('product', axis=1)

target_0 = geo_data_0['product']
target_1 = geo_data_1['product']
target_2 = geo_data_2['product']

Разделим данные на обучающие и валидационные выборки в соотношении 75:25

In [7]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

Стандартизируем признаки в выборках

In [8]:
scaler = StandardScaler()
pd.options.mode.chained_assignment = None 

def skaler_features(features_train, features_valid):
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

skaler_features(features_train_0, features_valid_0)
skaler_features(features_train_1, features_valid_1)
skaler_features(features_train_2, features_valid_2)

Обучим модели а так же найдем средний запас предсказаного сырья и RMSE моделей

In [9]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0)
predicted_mean_0 = predicted_valid_0.mean()
mse_0 = mean_squared_error(target_valid_0, predicted_valid_0)
print('Средний запас предсказаного сырья =', predicted_mean_0,
     '\nRMSE =', mse_0 ** 0.5)

Средний запас предсказаного сырья = 92.59256778438038 
RMSE = 37.5794217150813


In [10]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)
predicted_mean_1 = predicted_valid_1.mean()
mse_1 = mean_squared_error(target_valid_1, predicted_valid_1)
print('Средний запас предсказаного сырья =', predicted_mean_1,
     '\nRMSE =', mse_1 ** 0.5)

Средний запас предсказаного сырья = 68.728546895446 
RMSE = 0.893099286775616


In [11]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)
predicted_mean_2 = predicted_valid_2.mean()
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
print('Средний запас предсказаного сырья =', predicted_mean_2,
     '\nRMSE =', mse_2 ** 0.5)

Средний запас предсказаного сырья = 94.96504596800489 
RMSE = 40.02970873393434


**Вывод:** Модели 0 и 2 предсказывают больший средний запас сырья, но при этом у 2й модели очень низкий показатель RMSE, что говорит о хорошем уровне предсказаний модели

## Подготовка к расчёту прибыли

Подготовим переменные для расчета прибыли

In [12]:
POINTS = 500
FINAL_POINTS = 200
REGION_BUDGET = 10*(10**9)
INCOME = 450000

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [13]:
NO_DAMAGES_POINT = REGION_BUDGET / INCOME / FINAL_POINTS
NO_DAMAGES_POINT

111.11111111111111

Сравним его с предсказаниями наших моделей

In [14]:
for index, i in enumerate([predicted_mean_0, predicted_mean_1, predicted_mean_2], start=0): 
    if i > NO_DAMAGES_POINT:
        print(f'Предсказаный средний обьем сырья в регионе {index} прибыльный')
    else:
        print(f'Предсказаный средний обьем сырья в регионе {index} убыточный')

Предсказаный средний обьем сырья в регионе 0 убыточный
Предсказаный средний обьем сырья в регионе 1 убыточный
Предсказаный средний обьем сырья в регионе 2 убыточный


**Вывод:** мы подготовили данные для дальнейших расчетов и расчитали обьем сырья для безубыточной разработки сырья. По результатам сравнения с предсказаниями моделей оказалось, что предсказанный средний обьем во всех трех регионах убыточный

Напишем функцию для расчета прибыли по выбраным скважинам

In [15]:
def income_region(predicted_values, target_values):
    predicted_sort = pd.Series(data=predicted_values).sort_values(ascending=False)[:FINAL_POINTS]
    target_sort = (target_values.reset_index(drop = True)[predicted_sort.index])
    sum_target = target_sort.sum()
    income_200 = round((sum_target * INCOME - REGION_BUDGET))
    return income_200 

## Расчёт прибыли и рисков 

Напишем функцию с техникой bootstrap для подсчета доверительного интервала региона и риска убытков

In [16]:
state = np.random.RandomState(12345)
def risks(predicted_valid, target_valid):
    values = []
    for i in range(1000):
        predicted_subsample = pd.Series(data=predicted_valid).sample(n=POINTS, replace=True, random_state=state)
        values.append(income_region(predicted_subsample, target_valid))

    values = pd.Series(values)
    lower =  round(values.quantile(0.025))
    upper =  round(values.quantile(0.975))
    mean = values.mean()
    risk = values.apply(lambda x: x < 0).sum()/len(values)*100

    print("Средняя прибыль региона:", round(mean))
    print(f"Доверительный интервал региона: от {lower} до {upper}")
    print(f"Риск убытков {risk}%")

Применим функцию к регионам

In [17]:
print("Регион 0", "_"*51)
risks(predicted_valid_0, target_valid_0)
print("\nРегион 1", "_"*51)
risks(predicted_valid_1, target_valid_1)
print("\n"
      "\nРегион 2", "_"*51)
risks(predicted_valid_2, target_valid_2)

Регион 0 ___________________________________________________
Средняя прибыль региона: 396164985
Доверительный интервал региона: от -111215546 до 909766941
Риск убытков 6.9%

Регион 1 ___________________________________________________
Средняя прибыль региона: 461155817
Доверительный интервал региона: от 78050810 до 862952060
Риск убытков 0.7000000000000001%


Регион 2 ___________________________________________________
Средняя прибыль региона: 392950475
Доверительный интервал региона: от -112227626 до 934562915
Риск убытков 6.5%


Регионы с номерами 0 и 2 превышают возможные риски, у региона с номером 1 риски меньше 2.5 процентов, что удовлетворяет усовию, так же он имеет самую высокую среднюю прибыль.

## Общий вывод

В ходе исследовании мы изучили и подготовили данные для обучения моделей предсказания добычи сырья, обучили модели и сделали на них предсказания. Далее мы нашли среднее кол-во сырья для безубыточной разработки скважин и сравнили их с средними значениями по регионам, в результате чего обнаружили что среднее кол-во сырья в регионах убыточное, что указывает на необходимость хорошей модели для предсказания скважин. Далее мы рассчитали среднюю прибыль, доверительные интервалы и риски убытков по трём регионам.
В результате исследования мы нашли потенциальный регион для разработки новых скважин, им стал регион с номером 1, т.к. риск убытков в данном регионе меньше 2.5 процентов и он имеет наибольшую среднюю прибыль.